# Smol agents

These agents try to solve problems using code and the tools you provide - they can be used in similar fashion as any other code and orchestrated as such   

## install and helper methods

In [ ]:
%pip install --upgrade -q pip
%pip install -q ipywidgets
%pip install -q "smolagents[toolkit]" 
%pip install -q "smolagents[litellm]"
%pip install -q python-dotenv
%pip install -q "smolagents[e2b]"

from smolagents import ToolCallingAgent, WebSearchTool, VisitWebpageTool, tool, CodeAgent


#### System prompt

In [ ]:
better_prompt = '''
    You are an expert assistant who can solve any task using code blobs. You will be given a task to solve as best you can.
    To do so, you have been given access to a list of tools: these tools are basically Python functions which you can call with code.
    To solve the task, you must plan forward to proceed in a series of steps, in a cycle of 'Thought:', 'Code:', and 'Observation:' sequences.

    At each step, in the 'Thought:' attribute, you should first explain your reasoning towards solving the task and the tools that you want to use.
    Then in the 'Code' attribute, you should write the code in simple Python.
    During each intermediate step, you can use 'print()' to save whatever important information you will then need.
    These print outputs will then appear in the 'Observation:' field, which will be available as input for the next step.
    In the end you have to return a final answer using the `final_answer` tool. You will be generating a JSON object with the following structure:
    ```json
    {
    "thought": "...",
    "code": "..."
    }
    ```

    Here are a few examples using notional tools:
    ---
    Task: "Generate an image of the oldest person in this document."

    {"thought": "I will proceed step by step and use the following tools: `document_qa` to find the oldest person in the document, then `image_generator` to generate an image according to the answer.", "code": "answer = document_qa(document=document, question=\"Who is the oldest person mentioned?\")\nprint(answer)\n"}
    Observation: "The oldest person in the document is John Doe, a 55 year old lumberjack living in Newfoundland."

    {"thought": "I will now generate an image showcasing the oldest person.", "code": "image = image_generator(\"A portrait of John Doe, a 55-year-old man living in Canada.\")\nfinal_answer(image)\n"}
    ---
    Task: "What is the result of the following operation: 5 + 3 + 1294.678?"

    {"thought": "I will use python code to compute the result of the operation and then return the final answer using the `final_answer` tool", "code": "result = 5 + 3 + 1294.678\nfinal_answer(result)\n"}

    ---
    Task:
    In a 1979 interview, Stanislaus Ulam discusses with Martin Sherwin about other great physicists of his time, including Oppenheimer.
    What does he say was the consequence of Einstein learning too much math on his creativity, in one word?

    {"thought": "I need to find and read the 1979 interview of Stanislaus Ulam with Martin Sherwin.", "code": "pages = web_search(query=\"1979 interview Stanislaus Ulam Martin Sherwin physicists Einstein\")\nprint(pages)\n"}
    Observation:
    No result found for query "1979 interview Stanislaus Ulam Martin Sherwin physicists Einstein".

    {"thought": "The query was maybe too restrictive and did not find any results. Let's try again with a broader query.", "code": "pages = web_search(query=\"1979 interview Stanislaus Ulam\")\nprint(pages)\n"}
    Observation:
    Found 6 pages:
    [Stanislaus Ulam 1979 interview](https://ahf.nuclearmuseum.org/voices/oral-histories/stanislaus-ulams-interview-1979/)

    [Ulam discusses Manhattan Project](https://ahf.nuclearmuseum.org/manhattan-project/ulam-manhattan-project/)

    (truncated)

    {"thought": "I will read the first 2 pages to know more.", "code": "for url in [\"https://ahf.nuclearmuseum.org/voices/oral-histories/stanislaus-ulams-interview-1979/\", \"https://ahf.nuclearmuseum.org/manhattan-project/ulam-manhattan-project/\"]:\n      whole_page = visit_webpage(url)\n      print(whole_page)\n      print(\"\n\" + \"=\"*80 + \"\n\")  # Print separator between pages"}

    Observation:
    Manhattan Project Locations:
    Los Alamos, NM
    Stanislaus Ulam was a Polish-American mathematician. He worked on the Manhattan Project at Los Alamos and later helped design the hydrogen bomb. In this interview, he discusses his work at
    (truncated)

    {"thought": "I now have the final answer: from the webpages visited, Stanislaus Ulam says of Einstein: \"He learned too much mathematics and sort of diminished, it seems to me personally, it seems to me his purely physics creativity.\" Let's answer in one word.", "code": "final_answer(\"diminished\")"}

    ---
    Task: "Which city has the highest population: Guangzhou or Shanghai?"

    {"thought": "I need to get the populations for both cities and compare them: I will use the tool `web_search` to get the population of both cities.", "code": "for city in [\"Guangzhou\", \"Shanghai\"]:\n      print(f\"Population {city}:\", web_search(f\"{city} population\")"}
    Observation:
    Population Guangzhou: ['Guangzhou has a population of 15 million inhabitants as of 2021.']
    Population Shanghai: '26 million (2019)'

    {"thought": "Now I know that Shanghai has the highest population.", "code": "final_answer(\"Shanghai\")"}

    ---
    Task: "What is the current age of the pope, raised to the power 0.36?"

    {"thought": "I will use the tool `wikipedia_search` to get the age of the pope, and confirm that with a web search.", "code": "pope_age_wiki = wikipedia_search(query=\"current pope age\")\nprint(\"Pope age as per wikipedia:\", pope_age_wiki)\npope_age_search = web_search(query=\"current pope age\")\nprint(\"Pope age as per google search:\", pope_age_search)"}
    Observation:
    Pope age: "The pope Francis is currently 88 years old."

    {"thought": "I know that the pope is 88 years old. Let's compute the result using python code.", "code": "pope_current_age = 88 ** 0.36\nfinal_answer(pope_current_age)"}

    Above example were using notional tools that might not exist for you. On top of performing computations in the Python code snippets that you create, you only have access to these tools, behaving like regular python functions:
    ```python
    def final_answer(answer: any) -> any:
        """Provides a final answer to the given problem.

        Args:
            answer: The final answer to the problem
        """

    ```

    Here are the rules you should always follow to solve your task:
    1. Use only variables that you have defined!
    2. Always use the right arguments for the tools. DO NOT pass the arguments as a dict as in 'answer = wikipedia_search({'query': "What is the place where James Bond lives?"})', but use the arguments directly as in 'answer = wikipedia_search(query="What is the place where James Bond lives?")'.
    3. Take care to not chain too many sequential tool calls in the same code block, especially when the output format is unpredictable. For instance, a call to wikipedia_search has an unpredictable return format, so do not have another tool call that depends on its output in the same block: rather output results with print() to use them in the next block.
    4. Call a tool only when needed, and never re-do a tool call that you previously did with the exact same parameters.
    5. Don't name any new variable with the same name as a tool: for instance don't name a variable 'final_answer'.
    6. Never create any notional variables in our code, as having these in your logs will derail you from the true variables.
    7. You can use imports in your code, but only from the following list of modules: ['collections', 'datetime', 'itertools', 'math', 'queue', 'random', 're', 'stat', 'statistics', 'time', 'unicodedata']
    8. The state persists between code executions: so if in one step you've created variables or imported modules, these will all persist.
    9. Don't give up! You're in charge of solving the task, not providing directions to solve it.

    Now Begin!
'''


#### Helper Methods

In [ ]:
# helper methods
import os
from smolagents import OpenAIServerModel, LiteLLMModel
from dotenv import load_dotenv
load_dotenv()

def set_agent_prompt(
        agent, 
        type_of_prompt:str = "system_prompt", 
        prompt:str =    
            """Your a simple agent trying to solve problems using python code.
            You have access to a simple python interpreter, and potentially tools,
            when you have an answer use the final_answer tool 
            """
):
    agent.prompt_templates[type_of_prompt] = prompt

local_model_id = "ollama_chat/qwen2.5-coder:7b-instruct-q8_0"
default_api_base = "https://api.openai.com/v1"
default_model_id = "gpt-4.1-nano"
    
def load_model(model_id: str = default_model_id, api_base: str = default_api_base):

    
    api_key = os.environ.get("OPENAI_API_KEY")
    if not model_id:
        return OpenAIServerModel(
            model_id=model_id,
            api_base=default_api_base,
            api_key=api_key,

        )
    elif model_id.lower() == "local":
        return LiteLLMModel(
            model_id=local_model_id,
            api_base="http://localhost:11434",
            num_ctx=16384,
        )
    else: 
        return OpenAIServerModel(
            model_id=model_id,
            api_base=api_base,
            api_key=api_key,
        )  
    

model = load_model()

## Simple Processor

In [ ]:
# simple processor
from smolagents import CodeAgent
code_agent = CodeAgent(
    tools=[], 
    model=model,
    name="simple_processor",
    description="Your a Simple processor, acting as a chatbot serving chat input, you should answer in plan text",
    verbosity_level=2, # only result, 1 = show steps and code executed, 2 = also shows output from LLM 
    stream_outputs=True, # Streams the thoughts of the agent
    use_structured_outputs_internally=True, # Helps formatting the output in between steps improve performance for many models / Having a format allows for a better solution 
    # max_steps=3,
)


# set_agent_prompt(code_agent, prompt=better_prompt)
query = input()

code_agent.run(query)
# code_agent.memory.steps.append(te)

# code_agent.replay()

## Tool calling

### Simple tools

In [ ]:
# Tool Calling (Augmented LLM & Tool utility)
from smolagents import ToolCallingAgent, WebSearchTool, VisitWebpageTool, tool

@tool
def this_is_a_cool_tool(type_of_quote: str = "") -> str:
    """
    Provides the caller with a cool or inspiring quote.

    Args:
        type_of_quote (str): The type of quote to provide. Examples include "inspiring", "funny", "motivational", etc.
        
    Returns:
        A string containing the cool or inspiring quote.
    """

    return "This is an inspiring quote"



tool_agent = ToolCallingAgent(
    tools=[WebSearchTool(), VisitWebpageTool(), this_is_a_cool_tool], 
    model=model,
    name="tool_agent",
    description="Your a tool calling agent that runs web queries, can visit websites and provide inspiring quotes",
    # verbosity_level=2,
    # stream_outputs=True,
    max_steps=3

)

query = input()
if not query.strip():
    query = "Give me an inspiring quote"
 
tool_agent.run(query)

### Advanced tooling

In [ ]:
%pip install -q python-dotenv

import json
import os
from smolagents import Tool
from huggingface_hub import login
token = os.environ["HF_WRITE_API_KEY"]

login(token)

# download tools fra spaces

image_generation_tool = Tool.from_space(
    "black-forest-labs/FLUX.1-schnell",
    name="image_generator",
    description="Generate an image from a prompt"
)
tools = [WebSearchTool(), VisitWebpageTool(),image_generation_tool] 

# quotes_tool = load_tool("m-ric/Quotes", trust_remote_code=True)

tool_agent = ToolCallingAgent(
    tools=tools, 
    model=model,
    name="tool_agent",
    description="Your a tool calling agent that runs web queries, can visit websites and use other tools",
    max_steps=3
)

# query = input("what image would you like to generate")
# tool_agent.run(f"can you generate me an image of a {query} and put it into the 'image' folder")



%pip install -q google-api-python-client
from googleapiclient.discovery import build
import os

class YouTubeSearchTool(Tool):
    name = "youtube_search"
    description = (
        "Searches YouTube for videos matching a query. "
        "Returns the top video titles and URLs."
    )
    inputs = {
        "query": {
            "type": "string",
            "description": "Search term for querying YouTube videos",
        }
    }
    output_type = "string"  


    def setup(self):
        # Called once per tool instance, useful for setting up API key etc.
        # You might load it from env var or config
        load_dotenv()
        self.api_key = os.getenv("YOUTUBE_API_KEY")
        self.youtube = build("youtube", "v3", developerKey=self.api_key)

    def forward(self, query: str):
        req = self.youtube.search().list(q=query, part="snippet", type="video", maxResults=5)
        res = req.execute()
        lst = [
            {
                "title": item["snippet"]["title"],
                "url": f"https://www.youtube.com/watch?v={item['id']['videoId']}",
            }
            for item in res["items"]
        ]
        output = json.dumps(lst)
        return output

yt_search = YouTubeSearchTool()
tool_agent.tools[yt_search.name] = yt_search 


query = input("What youtuber do you want to find")
if query:
    tool_agent.run(f"Find the latest video and also the most popular video from {query}")


## Multi step agent - Human in the loop

In [ ]:
# Multi-Step** (Reasoning in time or steps, No over path)
from smolagents import CodeAgent, PlanningStep, DuckDuckGoSearchTool
def interrupt_after_plan(memory_step, agent):
    if isinstance(memory_step, PlanningStep):
        print("Agent's plan: " + str(agent.name))
        print(memory_step.plan)
        user_feedback = input("Do you want to modify the plan or continue? (Type 'modify' to edit, anything else to continue): ")
        if user_feedback.strip().lower() == "modify":
            new_plan = input("Enter your modified plan: ")
            memory_step.plan = new_plan
            print("Plan updated.")
            # Loop until user confirms the updated plan
            while True:
                print("Updated plan:")
                print(memory_step.plan)
                confirm = input("Do you confirm this updated plan? (Type 'yes' to confirm, anything else to edit again): ")
                if confirm.strip().lower() == "yes":
                    break
                new_plan = input("Enter your modified plan: ")
                memory_step.plan = new_plan
                print("Plan updated.")
        else:
            print("Continuing with current plan.")


multi_step = CodeAgent(
    model=model,
    tools=[DuckDuckGoSearchTool()],
    name="multi_step_agent",
    description="Multi step that solves task, you make a plan and also ask the human for feedback",
    planning_interval=3,  # Plan every 3 steps
    step_callbacks={PlanningStep: interrupt_after_plan},  # Register callback for PlanningStep
    max_steps=10,
)
task = input()

multi_step.run(task, reset=False)  # Keeps all previous steps, NB: also from previous runs

print(f"Current memory contains {len(multi_step.memory.steps)} steps:")
for i, step in enumerate(multi_step.memory.steps):
    step_type = type(step).__name__
    print(f"  {i+1}. {step_type}")


## Multi-agent

In [ ]:
# Multi-Agent (One agent, orchestrates ithers another agent)

web_agent = ToolCallingAgent(
    tools=[VisitWebpageTool(), WebSearchTool()],
    model=model,
    name="web_agent",
    description="Search the web and visit webpages",
    verbosity_level=0,
    stream_outputs=True,
    max_steps=10
)

code_agent = CodeAgent(
    tools=[], 
    model=model,
    name="code_agent",
    description="You are a managing agent which can solve problems but also use managede agents",
    # verbosity_level=0,
    # stream_outputs=True,
    use_structured_outputs_internally=True,
    managed_agents=[web_agent],
    max_steps=10
)

query = input()
code_agent.run(query)

replay = input("Would you like to replay the main agent's steps? (Press Enter to skip): ")
if replay:
    code_agent.replay()

replay = input("Would you like to replay the web agent's steps? (Press Enter to skip): ")
if replay:
    code_agent.managed_agents["web_agent"].replay()


## Augmented LLM

### Setup RAG tool

In [ ]:
%pip install -q smolagents pandas langchain langchain-community sentence-transformers datasets python-dotenv rank_bm25 --upgrade

import datasets
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever

# Load the Hugging Face documentation dataset
knowledge_base = datasets.load_dataset("m-ric/huggingface_doc", split="train")

# Filter to include only Transformers documentation
knowledge_base = knowledge_base.filter(lambda row: row["source"].startswith("huggingface/transformers"))

# Convert dataset entries to Document objects with metadata
source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"].split("/")[1]})
    for doc in knowledge_base
]

# Split documents into smaller chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Characters per chunk
    chunk_overlap=50,  # Overlap between chunks to maintain context
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],  # Priority order for splitting
)
docs_processed = text_splitter.split_documents(source_docs)

print(f"Knowledge base prepared with {len(docs_processed)} document chunks")


from smolagents import Tool

class RetrieverTool(Tool):
    name = "retriever"
    description = "Uses semantic search to retrieve the parts of transformers documentation that could be most relevant to answer your query."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        # Initialize the retriever with our processed documents
        self.retriever = BM25Retriever.from_documents(
            docs, k=10  # Return top 10 most relevant documents
        )

    def forward(self, query: str) -> str:
        """Execute the retrieval based on the provided query."""
        assert isinstance(query, str), "Your search query must be a string"

        # Retrieve relevant documents
        docs = self.retriever.invoke(query)

        # Format the retrieved documents for readability
        return "\nRetrieved documents:\n" + "".join(
            [
                f"\n\n===== Document {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

### Augmented LLM

In [ ]:
# Initialize our retriever tool with the processed documents
retriever_tool = RetrieverTool(docs_processed)

agent = CodeAgent(
    tools=[retriever_tool],  
    model=model,  
    max_steps=4, 
    verbosity_level=2,  # Show detailed agent reasoning
)

# Ask a question that requires retrieving information
question = "For a transformers model training, which is slower, the forward or the backward pass?"

# Run the agent to get an answer
agent_output = agent.run(question)

# Display the final answer
print("\nFinal answer:")
print(agent_output)

## Fully Autonomous

In [ ]:
# Fully Autonomous Agent** (No fixed path, no fixed Steps)
%pip install -q google-search-results
from smolagents import ToolCollection, tools
from langchain.agents import load_tools

# from langchain_community.agent_toolkits.load_tools import get_all_tool_names

# tool_names = get_all_tool_names()
# print(tool_names)


load_dotenv()

tool = load_tools(["metaphor-search"])[0]

temp_tool = Tool.from_langchain(tool)

model = load_model()
tools = []
tools.append(temp_tool)
code_agent = CodeAgent(
    model=model, 
    tools=tools, 
    use_structured_outputs_internally=True,
    stream_outputs=True,
    max_steps=10
)
# set_agent_prompt(code_agent)

query = "Give me a metaphor"
code_agent.run(query)

# Access til nye tools

search_tool = Tool.from_langchain(load_tools(["serpapi"])[0])

code_agent.tools[search_tool.name] = search_tool
print(search_tool.name)


code_agent.run("Can you find the best restaraunts to bring a date in Copenhagen, it should be affordable, also help me plan activities for before and after")

problem = input()
if problem:
    query = f"Find the best tool on langchain or hugginface to solve {problem}, output it such that i can be input directly into Tool.from_langchain(load_tools(['YouTubeSearch'])[0])"
    tool = code_agent.run(query)
    code_agent.tools[tool.name] = tool


use_tool = input()
if use_tool:
    code_agent.run(use_tool)

    


## Workflow

In [ ]:
# - **Workflow** (Multi-step reasoning, outside of workflow)



## Extra features

An example from smolagents repo, trying to replicate the deep reasearch feature. This agent achieves 55% pass@1 on the GAIA validation set, compared to 67% for the original Deep Research.
- https://github.com/huggingface/smolagents/blob/83ff2f7ed09788e130965349929f8bd5152e507e/examples/open_deep_research/run.py


### Multi-step ui interactive

Doesn't run in jupyter notebook, run gradiotest.py file insteads

In [ ]:
%pip install -q "gradio"
%pip install -q 'smolagents[gradio]'
%pip install --upgrade -q websockets
%pip show gradio
# %gradio  app.py hot reload 
# %gradio --vibe app.py, allow for text editor in the browser

from smolagents import CodeAgent, GradioUI, load_tool
image_generation_tool = load_tool("m-ric/text-to-image", trust_remote_code=True)

model = load_model()
agent = CodeAgent(tools=[image_generation_tool], model=model)
# gradio_ui = GradioUI(
#     agent, 
#     file_upload_folder="./uploads", 
#     reset_agent_memory=False # Set to true to agent memory inbetween interactions
# ) 
# gradio_ui.launch(inline=True) # Inline should allow for running in a jupyter notebook

### Load and push agents

In [ ]:
from huggingface_hub import login
token = os.environ["HF_WRITE_API_KEY"]

login(token)

# # Load agents 

# agent = CodeAgent.from_hub("AgenticSolutions/character_roleplay_chatbot", trust_remote_code=True)


# Upload agent 
code_agent.push_to_hub("Thomasitu/thesis", private=True)
# Repo: https://huggingface.co/spaces/Thomasitu/thesis/tree/main 
# Laver en Agent.JSON, gemmer configuration


### Adjust agent memory, reuse, memory from other agents etc

In [ ]:
from smolagents import InferenceClientModel, CodeAgent, ActionStep, TaskStep

agent = CodeAgent(tools=[], model=InferenceClientModel(), verbosity_level=1)
agent.python_executor.send_tools({**agent.tools})
print(agent.memory.system_prompt)

task = "What is the 20th Fibonacci number?"

# You could modify the memory as needed here by inputting the memory of another agent.
# agent.memory.steps = previous_agent.memory.steps

# Let's start a new task!
agent.memory.steps.append(TaskStep(task=task, task_images=[]))

final_answer = None
step_number = 1
while final_answer is None and step_number <= 10:
    memory_step = ActionStep(
        step_number=step_number,
        observations_images=[],
    )
    # Run one step.
    final_answer = agent.step(memory_step)
    agent.memory.steps.append(memory_step)
    step_number += 1

    # Change the memory as you please!
    # For instance to update the latest step:
    # agent.memory.steps[-1] = ...

print("The final answer is:", final_answer)

## Secure environment

#### E2B Sandbox


##### Single agent

In [ ]:
from smolagents import CodeAgent
import os
load_dotenv()

api_key = os.getenv("E2B_API_KEY")

model = load_model()
with CodeAgent(model=model, tools=[], executor_type="e2b",executor_kwargs=({"api_key": os.getenv("E2B_API_KEY")})) as agent:
    agent.run("Can you give me the 100th Fibonacci number?")

##### Multi-agent

In [ ]:
from e2b_code_interpreter import Sandbox
import os
load_dotenv()

api_key = os.getenv("E2B_API_KEY")
# Create the sandbox
sandbox = Sandbox(api_key=api_key)

# Install required packages
sandbox.commands.run("pip install smolagents")
sandbox.commands.run("pip install openai")

def run_code_raise_errors(sandbox, code: str, verbose: bool = False) -> str:
    execution = sandbox.run_code(
        code,
        envs={'OPENAI_API_KEY': os.getenv('OPENAI_API_KEY')}
    )
    if execution.error:
        execution_logs = "\n".join([str(log) for log in execution.logs.stdout])
        logs = execution_logs
        logs += execution.error.traceback
        raise ValueError(logs)
    return "\n".join([str(log) for log in execution.logs.stdout])

# Define your agent application
agent_code = """
import os
from smolagents import CodeAgent, OpenAIServerModel


default_api_base = 'https://api.openai.com/v1'
default_model_id = 'gpt-4.1-nano'
token = os.getenv('OPENAI_API_KEY')

model = OpenAIServerModel(
            model_id=default_model_id,
            api_base=default_api_base,
            api_key=token
        )


# Initialize the agents
agent = CodeAgent(
    model=model,
    tools=[],
    name="coder_agent",
    description="This agent takes care of your difficult algorithmic problems using code."
)

manager_agent = CodeAgent(
    model=model,
    tools=[],
    managed_agents=[agent],
)

# Run the agent
response = manager_agent.run("What's the 20th Fibonacci number?")
print(response)
"""

# Run the agent code in the sandbox
execution_logs = run_code_raise_errors(sandbox, agent_code)
print(execution_logs)

#### Docker

##### Single agent

In [ ]:
%pip install -q 'smolagents[docker]'
from smolagents import CodeAgent

with CodeAgent(model=model, tools=[], executor_type="docker") as agent:
    agent.run("Can you give me the 100th Fibonacci number?")


##### Multi-agent 

In [ ]:
import docker
import os
from typing import Optional


class DockerSandbox:
    def __init__(self):
        self.client = docker.from_env()
        self.container = None

    def create_container(self):
        try:
            image, build_logs = self.client.images.build(
                path=".",
                tag="agent-sandbox",
                rm=True,
                forcerm=True,
                buildargs={},
                # decode=True
            )
        except docker.errors.BuildError as e:
            print("Build error logs:")
            for log in e.build_log:
                if 'stream' in log:
                    print(log['stream'].strip())
            raise

        # Create container with security constraints and proper logging
        self.container = self.client.containers.run(
            "agent-sandbox",
            command="tail -f /dev/null",  # Keep container running
            detach=True,
            tty=True,
            mem_limit="512m",
            cpu_quota=50000,
            pids_limit=100,
            security_opt=["no-new-privileges"],
            cap_drop=["ALL"],
            environment={
                "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY")
            },
        )

    def run_code(self, code: str) -> Optional[str]:
        if not self.container:
            self.create_container()

        # Execute code in container
        exec_result = self.container.exec_run(
            cmd=["python", "-c", code],
            user="nobody"
        )

        # Collect all output
        return exec_result.output.decode() if exec_result.output else None


    def cleanup(self):
        if self.container:
            try:
                self.container.stop()
            except docker.errors.NotFound:
                # Container already removed, this is expected
                pass
            except Exception as e:
                print(f"Error during cleanup: {e}")
            finally:
                self.container = None  # Clear the reference

# Example usage:
sandbox = DockerSandbox()

import os
load_dotenv()

# Create the sandbox

# Install required packages
sandbox.run_code("pip install smolagents")
sandbox.run_code("pip install smolagents[openai]")


try:
    # Define your agent code
    agent_code = """

import os
from smolagents import CodeAgent, OpenAIServerModel 

default_api_base = 'https://api.openai.com/v1'
default_model_id = 'gpt-4.1-nano'
token = os.getenv('OPENAI_API_KEY')

model = OpenAIServerModel(
            model_id=default_model_id,
            api_base=default_api_base,
            api_key=token
        )
# Initialize the agent
agent = CodeAgent(
    model=model,
    tools=[]
)

# Run the agent
response = agent.run("What's the 20th Fibonacci number?")
print(response)
"""

    # Run the code in the sandbox
    output = sandbox.run_code(agent_code)
    print(output)

finally:
    sandbox.cleanup()

## MCP

### Setup local MCP Server

In [ ]:
"""
FastMCP Desktop Example

A simple example that exposes the desktop directory as a resource.
"""

from pathlib import Path

from mcp.server.fastmcp import FastMCP

# Create server
mcp = FastMCP("Demo")


@mcp.resource("dir://desktop")
def desktop() -> list[str]:
    """List the files in the user's desktop"""
    desktop = Path.home() / "Desktop"
    return [str(f) for f in desktop.iterdir()]


@mcp.tool()
def sum(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

if __name__ == "__main__":
    mcp.run()


### Handle lifecycle of MCP client automaticly

In [ ]:
# %pip install -q mcp
%pip install -q 'smolagents[mcp]'
from smolagents import MCPClient, CodeAgent
import os
from mcp import StdioServerParameters


load_dotenv()
server_parameters = StdioServerParameters(
    command="uvx",  # Using uvx ensures dependencies are available
    args=["--quiet", "pubmedmcp@0.1.3"],
    env={"UV_PYTHON": "3.12", **os.environ},
)

model = load_model()

with MCPClient(server_parameters) as tools:
    agent = CodeAgent(tools=tools, model=model, use_structured_outputs_internally=True, max_steps=10)
    
    query = input("Enter your query: ")
    if query:
        agent.run("Please find the latest research on COVID-19 treatment.")

    replay = input("Do you want to replay the steps? (Y/'')")
    if replay:
        agent.replay()

# local server:
with MCPClient({"url": "http://localhost:8000/sse", "transport": "sse"}) as tools:
    code_agent = CodeAgent(
        model=model,
        tools=tools, 
        name="code_agent",
        description="",
        max_steps=10,
        use_structured_outputs_internally=True
    )
    
    query = input("Enter your query: ")
    code_agent.run(query) if query else None

    replay = input("Do you want to replay the steps? (Y/'')")
    if replay:
        agent.replay()

### Manual lifecycle

In [ ]:
from mcp import StdioServerParameters
import os
load_dotenv()

model = load_model()

try:
    mcp_client = MCPClient({"url": "http://localhost:8000/sse", "transport": "sse"})
    tools = mcp_client.get_tools()

    code_agent = CodeAgent()
    # use your tools here.
finally:
    mcp_client.disconnect()


### Use Multiple MCP Servers

In [ ]:
# Initialize server parameters
pubmed_server = StdioServerParameters(
    command="uvx",
    args=["--quiet", "pubmedmcp@0.1.3"],
    env={"UV_PYTHON": "3.12", **os.environ},
)

local_server = {"url": "http://127.0.0.1:8000/sse"}


# Manually manage the connection
try:
    mcp_client = MCPClient([pubmed_server, local_server])
    tools = mcp_client.get_tools()

    # Use the tools with your agent
    agent = CodeAgent(tools=tools, model=model)
    result = agent.run("What are the recent therapeutic approaches for Alzheimer's disease?")

    # Process the result as needed
    print(f"Agent response: {result}")
finally:
    # Always ensure the connection is properly closed
    mcp_client.disconnect()